# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

In [1]:
import random
import numpy as np

from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dists
import torch.utils.data as data

import torchvision.datasets as datasets
import torchvision.transforms as transforms

import syft

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

hook = syft.TorchHook(torch)

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [2]:
n_workers = 30

workers = [syft.VirtualWorker(hook, id="Worker:{:d}".format(i)) for i in range(n_workers)]
trusted_aggregator = syft.VirtualWorker(hook, id='Trusted Aggregator')

In [3]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset  = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

federated_mnist_trainset = mnist_trainset.federate(workers)

In [4]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()

        # 1x28x28
        self.conv0      = nn.Conv2d(1, 4, 3, padding=1)
        self.maxpool0   = nn.MaxPool2d(2)
        # 4x14x14
        self.conv1      = nn.Conv2d(4, 6, 3, padding=1)
        self.maxpool1   = nn.MaxPool2d(2)
        # 6x 7x 7
        self.conv2      = nn.Conv2d(6, 8, 3, padding=1)
        self.maxpool2   = nn.MaxPool2d(2, padding=1)
        # 8x 4x 4 = 128
        self.fc         = nn.Linear(128, 10)

        self.activation = nn.ReLU()
        
    def forward(self, x):
        x = self.conv0(x)
        x = self.activation(x)
        x = self.maxpool0(x)
        x = self.conv1(x)
        x = self.activation(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.maxpool2(x)
        x = self.fc(x.view(-1, 128))
        
        return x

## Averaged Gradients Approach (My Approach)

In [5]:
model = MNISTClassifier()

In [6]:
n_epochs   = 100
lr         = 2e-2
batch_size = 128

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(reduction='sum')

stored_grads = OrderedDict((name, param.new_zeros(param.shape).send(trusted_aggregator)) for name, param in model.named_parameters())
for i_epoch in range(n_epochs):
    print("Epoch {:d}:".format(i_epoch))
    
    optimizer.zero_grad()
    model = model.send(syft.local_worker)
    # Track the number of training examples in order to average the gradients later (+ additional stats)
    instance_count = torch.tensor(0.).send(trusted_aggregator)
    running_loss   = torch.tensor(0.).send(trusted_aggregator)
    correct_count  = torch.tensor(0.).send(trusted_aggregator)
    for i, worker in enumerate(workers):
        print("worker {:d}/{:d}".format(i, len(workers)-1), end='\r')
        model.move(worker)

        dataset = federated_mnist_trainset.datasets[worker.id]
        dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        imgs, labels = next(iter(dataloader))

        preds = model(imgs)

        loss = criterion(preds, labels)
        loss.backward()
        
        current_instance_count = torch.tensor(imgs.shape[0]).send(worker)
        current_loss           = loss.data.clone()
        current_correct_count  = torch.sum(torch.eq(preds.data.argmax(dim=1), labels))
        
        model.move(trusted_aggregator)
        current_instance_count.move(trusted_aggregator)
        current_loss.move(trusted_aggregator)
        current_correct_count.move(trusted_aggregator)

        for name, param in model.named_parameters():
            stored_grads[name].add_(param.grad)
            param.grad.zero_()
        instance_count.add_(current_instance_count)
        running_loss.add_(current_loss)
        correct_count.add_(current_correct_count)

    for name, param in model.named_parameters():
        param.grad.add_(torch.div(stored_grads[name], instance_count))
        stored_grads[name].zero_()

    model = model.get()

    avg_loss     = running_loss.div_(instance_count).get().item()
    avg_accuracy = correct_count.div_(instance_count).get().item()
    
    print()
    print("    Training Loss:", avg_loss)
    print("    Training Accuracy:", avg_accuracy)

    optimizer.step()

Epoch 0:
worker 29/29
    Training Loss: 2.309325695037842
    Training Accuracy: 0.08124999701976776
Epoch 1:
worker 29/29
    Training Loss: 2.297940731048584
    Training Accuracy: 0.12994791567325592
Epoch 2:
worker 29/29
    Training Loss: 2.2785115242004395
    Training Accuracy: 0.3033854067325592
Epoch 3:
worker 29/29
    Training Loss: 2.246328115463257
    Training Accuracy: 0.3213541805744171
Epoch 4:
worker 29/29
    Training Loss: 2.179736375808716
    Training Accuracy: 0.3309895694255829
Epoch 5:
worker 29/29
    Training Loss: 2.0554521083831787
    Training Accuracy: 0.42005208134651184
Epoch 6:
worker 29/29
    Training Loss: 1.8949917554855347
    Training Accuracy: 0.500781238079071
Epoch 7:
worker 29/29
    Training Loss: 1.6481525897979736
    Training Accuracy: 0.6005208492279053
Epoch 8:
worker 29/29
    Training Loss: 1.3638745546340942
    Training Accuracy: 0.6244791746139526
Epoch 9:
worker 29/29
    Training Loss: 1.1782138347625732
    Training Accuracy: 0

    Training Accuracy: 0.9557291865348816
Epoch 80:
worker 29/29
    Training Loss: 0.12067414820194244
    Training Accuracy: 0.9611979126930237
Epoch 81:
worker 29/29
    Training Loss: 0.12396706640720367
    Training Accuracy: 0.964062511920929
Epoch 82:
worker 29/29
    Training Loss: 0.10720615833997726
    Training Accuracy: 0.9664062261581421
Epoch 83:
worker 29/29
    Training Loss: 0.13940660655498505
    Training Accuracy: 0.9557291865348816
Epoch 84:
worker 29/29
    Training Loss: 0.12093702703714371
    Training Accuracy: 0.9614583253860474
Epoch 85:
worker 29/29
    Training Loss: 0.12062196433544159
    Training Accuracy: 0.9598958492279053
Epoch 86:
worker 29/29
    Training Loss: 0.1198357492685318
    Training Accuracy: 0.9635416865348816
Epoch 87:
worker 29/29
    Training Loss: 0.12029659003019333
    Training Accuracy: 0.9604166746139526
Epoch 88:
worker 29/29
    Training Loss: 0.11127721518278122
    Training Accuracy: 0.9619791507720947
Epoch 89:
worker 29/29
 

In [7]:
test_dataloader = data.DataLoader(mnist_testset, batch_size=1024)

test_loss      = 0
instance_count = 0
correct_count  = 0
with torch.no_grad():
    for i, (imgs, labels) in enumerate(test_dataloader, 1):
        print("Batch {:d}/{:d}".format(i, len(test_dataloader)), end='\r')
        instance_count += imgs.size(0)

        preds = model(imgs)

        test_loss += criterion(preds, labels).item()
        correct_count += (preds.argmax(dim=1) == labels).sum().item()

print()
print("Test Loss:", test_loss / instance_count)
print("Test Accuracy:", correct_count / instance_count)

Batch 10/10
Test Loss: 0.09841546325683594
Test Accuracy: 0.9674


## Averaged Model Parameters Approach (Based on Andrew's Solution)

In [8]:
model = MNISTClassifier()

In [9]:
n_epochs   = 10
n_steps    = 30
lr         = 2e-2
batch_size = 128

local_models     = [model.copy().send(worker) for worker in workers]
local_optimizers = [optim.SGD(local_model.parameters(), lr=lr) for local_model in local_models]
criterion        = nn.CrossEntropyLoss()
test_dataloader  = data.DataLoader(mnist_testset, batch_size=1024)

for i_epoch in range(n_epochs):
    print("Epoch {:d}:".format(i_epoch))

    for i, (worker, local_model, local_optimizer) in enumerate(zip(workers, local_models, local_optimizers)):

        dataset = federated_mnist_trainset.datasets[worker.id]
        dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        loss = float('inf')
        for i_step in range(n_steps):
            print("worker {:d}/{:d} - Step {:d}/{:d} Loss={:.4f}       ".format(i, len(workers)-1, i_step, n_steps-1, loss), end='\r')

            imgs, labels = next(iter(dataloader))

            preds = local_model(imgs)

            local_optimizer.zero_grad()
            loss = criterion(preds, labels)
            loss.backward()
            local_optimizer.step()
            loss = loss.data.clone().get().item()

    model = model.send(trusted_aggregator)
    for global_param, local_params in zip(model.parameters(), zip(*[local_model.parameters() for local_model in local_models])):
        debug = global_param.data.clone().get()
        global_param.data.zero_()
        global_param.data.add_(torch.mean(torch.stack([local_param.data.clone().move(trusted_aggregator) for local_param in local_params]), dim=0))
        for local_param, worker in zip(local_params, workers):
            local_param.data.set_(global_param.data.clone().move(worker))

    model = model.get()

    test_loss      = 0
    instance_count = 0
    correct_count  = 0
    with torch.no_grad():
        for imgs, labels in test_dataloader:
            instance_count += imgs.size(0)

            preds = model(imgs)

            test_loss += criterion(preds, labels).item() * imgs.size(0)
            correct_count += (preds.argmax(dim=1) == labels).sum().item()

    print()
    print("    Test Loss:", test_loss / instance_count)
    print("    Test Accuracy:", correct_count / instance_count)

Epoch 0:
worker 29/29 - Step 29/29 Loss=2.2935       
    Test Loss: 2.2980876850128173
    Test Accuracy: 0.0868
Epoch 1:
worker 29/29 - Step 29/29 Loss=2.3009       
    Test Loss: 2.294271186828613
    Test Accuracy: 0.1528
Epoch 2:
worker 29/29 - Step 29/29 Loss=2.2947       
    Test Loss: 2.289651180267334
    Test Accuracy: 0.1844
Epoch 3:
worker 29/29 - Step 29/29 Loss=2.2800       
    Test Loss: 2.2833085285186767
    Test Accuracy: 0.2093
Epoch 4:
worker 29/29 - Step 29/29 Loss=2.2648       
    Test Loss: 2.2742213241577147
    Test Accuracy: 0.24
Epoch 5:
worker 29/29 - Step 29/29 Loss=2.2386       
    Test Loss: 2.259975256729126
    Test Accuracy: 0.2531
Epoch 6:
worker 29/29 - Step 29/29 Loss=2.2235       
    Test Loss: 2.2355145835876464
    Test Accuracy: 0.2522
Epoch 7:
worker 29/29 - Step 29/29 Loss=2.1236       
    Test Loss: 2.187098441696167
    Test Accuracy: 0.2788
Epoch 8:
worker 29/29 - Step 29/29 Loss=1.9480       
    Test Loss: 2.071344761657715
    Tes

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [10]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [11]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
print(decrypted_x)

5


Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [12]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
print(decrypted_x)

10


This even works for addition between two shared values!!

In [13]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
print(decrypted_z)

12


As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [14]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
print(alice_x_share)

23740629843736686616461


In [15]:
print((bob_x_share + alice_x_share) % Q)

5


So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [16]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [17]:
Q = 23740629843760239486723

def encrypt(x, n_shares):
    assert x < Q, "`x` is too large."
    shares = []

    for _ in range(n_shares - 1):
        shares.append(random.randint(0, Q))

    shares.append(Q - (sum(shares) % Q) + x)

#     random.shuffle(shares)

    return tuple(shares)

In [18]:
encrypt(7, 5)

(17411921852787415668783,
 9495176865499091630711,
 17880416036434995975129,
 9896138474794586231777,
 16538236301764628953776)

In [19]:
def decrypt(shares):
    return sum(shares) % Q

In [20]:
decrypt(encrypt(7, 5))

7

In [21]:
def add(shares_a, shares_b):
    assert len(shares_a) == len(shares_b), "Length mismatch between two shares."

    return tuple(a + b for a, b in zip(shares_a, shares_b))

In [22]:
decrypt(add(encrypt(173, 10), encrypt(141, 10)))

314

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [23]:
BASE=10
PRECISION=4

In [24]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [25]:
encode(3.5)

35000

In [26]:
decode(35000)

3.5

In [28]:
x = encrypt(encode(5.5), 3)
y = encrypt(encode(2.3), 3)
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
bob = syft.VirtualWorker(hook, id='bob')
alice = syft.VirtualWorker(hook, id='alice')
secure_worker = syft.VirtualWorker(hook, id='secure_worker')

bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [31]:
x = torch.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [32]:
x = x.share(bob, alice, secure_worker)

In [33]:
bob._objects

{52004417206: tensor([4607945012089164331,  933574258170101832, 3364821117505162012,
         2837592453351812753, 1788268197691109352])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x

In [35]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:39716819677 -> bob:87758319155]
	-> (Wrapper)>[PointerTensor | me:53980983548 -> alice:52966939567]
	-> (Wrapper)>[PointerTensor | me:8098695900 -> secure_worker:45927138476]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = torch.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = torch.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [45]:
y = x + x

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation